***
<font size="6"><center><b> HD-CapsNet: A Hierarchical Deep Capsule Network for Image Classification </b></center></font>
***

**Changing Model Architecture**
- **(Mod-2.1)** From coarse-to-Fine slowly decrease the dimension. i.e. 32D>16D>8D (Coarse>Medium>FINE), While keeping the primary capsule dimension same as original ($P_{capsule} = 8D$). Training with $L_{cons}$
    

# Files and Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras import backend as K

import tensorflow_docs as tfdocs
import tensorflow_docs.plots

# Supporting Libraries:
    #Mathplot lib for ploting graphs
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
    # numpy and pandas
import numpy as np
import pandas as pd
    #system
import os
import sys
import csv
    #import other libraries
import math
import random
from datetime import datetime
from treelib import Tree
    # ML model, Dataset and evalution metrics
sys.path.append('../../') ### adding system parth for src folder
from src import datasets # load datasets
from src import MixUp_add_loss # load datasets
from src import metrics # load hierarchcial metrics
from src import sysenv # load hierarchcial metrics
from src import models # load machine learning models

    ## Tensorflow_docs
import tensorflow_docs as tfdocs
import tensorflow_docs.plots

    # Auto reload local libraries if updated
%load_ext autoreload
%autoreload 2

# System information & GPU growth

In [2]:
systeminfo = sysenv.systeminfo()
print(systeminfo)


• Computer Name = SARUMAN
• Working Directory = D:\knoor\Deep Learning with Python\RunningOnServer\HD-CapsNet_MOD_2\Training_and_Analysis\4_CIFAR-100
• Python Version = 3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]
• TensorFlow Version = 2.8.0
• Keras Version = 2.8.0
• Current Environment = Anaconda Environment Name : D:\knoor\AnacondaEnvironment\py38tf28


In [3]:
gpus = "0,1,2,3,4,5,6,7" ## Selecting Available gpus (Multi-GPUS)
gpus = "4" ## Selecting Available gpus (Single GPU)
gpugrowth = sysenv.gpugrowth(gpus = gpus) ## Limiting GPUS from OS environment
gpugrowth.memory_growth() #GPU memory growth

Following GPUS are selected =  4
Done: GPU PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
1 Physical GPUs, 1 Logical GPUs


## log directory

In [4]:
directory = sysenv.log_dir('MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss')

FOLDER CREATED =  ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss


# Import Dataset : CIFAR-100

In [5]:
dataset = datasets.CIFAR100(version = 'ALL') # importing CIFAR10 Dataset
# dataset = datasets.CIFAR100(version = 'reduce') # importing CIFAR10 Dataset


CIFAR-100 dataset: Training have 50,000 samples and testing have 10,000 samples


# Parameters

In [6]:
train_params = {"n_epochs" : 100,
                "batch_size": 64,
                "lr": 0.001, # Initial learning rate
                "lr_decay": 0.95, # Learning rate decay
                "decay_exe": 9, #learning rate decay execution epoch after
               }
model_params = {"optimizer": tf.keras.optimizers.Adam(train_params['lr']),
                "loss_function": models.MarginLoss(),
               }

## Learning Rate Decay Scheduler

In [7]:
def scheduler(epoch):
    learning_rate_init = train_params["lr"]
    
    if epoch > train_params["decay_exe"]:
        learning_rate_init = train_params["lr"] * (train_params["lr_decay"] ** (epoch-9))
        
    tf.summary.scalar('learning rate', data=learning_rate_init, step=epoch)
        
    return learning_rate_init

# Bottom up Approach

## Loss Function

In [8]:
number_of_classes_c = len(np.unique(np.argmax(dataset['y_train_coarse'], axis=1)))
number_of_classes_m = len(np.unique(np.argmax(dataset['y_train_medium'], axis=1)))
number_of_classes_f = len(np.unique(np.argmax(dataset['y_train_fine'], axis=1)))

## For Dynamic LossWeights
initial_lw = models.initial_lw({"coarse": number_of_classes_c,
                                "medium": number_of_classes_m,
                                "fine": number_of_classes_f})

lossweight = {'coarse_lw' : K.variable(value = initial_lw['coarse'], dtype="float32", name="coarse_lw"),
             'medium_lw' : K.variable(value = initial_lw['medium'], dtype="float32", name="medium_lw"),
             'fine_lw' : K.variable(value = initial_lw['fine'], dtype="float32", name="fine_lw"),
              'decoder_lw' : 0.0
             }

In [9]:
coarse_to_medium_array = np.zeros(shape=[number_of_classes_c,number_of_classes_m], dtype=np.int32)
medium_to_fine_array = np.zeros(shape=[number_of_classes_m,number_of_classes_f], dtype=np.int32)

c_id = np.argmax(dataset['y_train_coarse'],1)

m_id = np.argmax(dataset['y_train_medium'],1)

f_id = np.argmax(dataset['y_train_fine'],1)

for x in range(len(dataset['y_test_medium'])):
    coarse_to_medium_array[c_id[x]][m_id[x]] = 1
    
for x in range(len(dataset['y_test_fine'])):
    medium_to_fine_array[m_id[x]][f_id[x]] = 1

Matrix_coarse_to_medium_OneHot = tf.constant(coarse_to_medium_array, dtype=tf.float32)
Matrix_medium_to_fine_OneHot = tf.constant(medium_to_fine_array, dtype=tf.float32)

In [10]:
def margin_loss(y_true, y_proba):
    
    present_error_raw = tf.square(tf.maximum(0., 0.9 - y_proba), name="present_error_raw")
    absent_error_raw = tf.square(tf.maximum(0., y_proba - 0.1), name="absent_error_raw")
    L = tf.add(y_true * present_error_raw, 0.5 * (1.0 - y_true) * absent_error_raw,name="L")
    total_marginloss = tf.reduce_sum(L, axis=1, name="margin_loss")

    return total_marginloss


def consistency_check(y_pred_ancestor,y_pred_current,lookup_matrix,num_class_current):
    pred_max_ancestor = tf.argmax(y_pred_ancestor,axis=1)
    pred_max_current = tf.argmax(y_pred_current,axis=1)
    
    consistant_check = tf.gather(lookup_matrix, indices=pred_max_ancestor)*tf.one_hot(pred_max_current,num_class_current)
    
    return tf.reduce_sum(consistant_check,1)

def get_consistency(y_true_ancestor, y_pred, lookup_matrix):
    '''
    Get consistency based on 2 levels
    Provide ture levels for the level above, predictions for the current level and a look up metrix
    '''
    y_prob = tf.math.divide(y_pred,tf.reshape(tf.reduce_sum(y_pred,-1),(-1,1),name='reshape'),name='Probability')
    
    index_for_predictions = tf.cast(tf.math.argmax(y_true_ancestor,axis=1),dtype= 'int32')
    consistent_fine = tf.gather(lookup_matrix, indices=index_for_predictions) * y_prob
    Consistency_sum_array = tf.reduce_sum(consistent_fine, axis =1)
    
    return tf.abs(1-Consistency_sum_array)

def CustomLoss(y_true_c, y_true_m, y_true_f, y_pred_c, y_pred_m, y_pred_f, LW_C, LW_M, LW_F,tree: Tree,C_Weight=0.2):
    
    con_m = consistency_check(y_pred_c,y_pred_m,Matrix_coarse_to_medium_OneHot,num_class_current=number_of_classes_m)
    con_m_not = tf.abs(con_m-1)
    
    con_f = consistency_check(y_pred_m,y_pred_f,Matrix_medium_to_fine_OneHot,num_class_current=number_of_classes_f)
    con_f_not = tf.abs(con_f-1)
    
    con_sum_m = get_consistency(y_true_c,y_pred_m,Matrix_coarse_to_medium_OneHot)
    con_sum_f = get_consistency(y_true_m,y_pred_f,Matrix_medium_to_fine_OneHot)
    
    medium_lvl_cosistency = con_sum_m * con_m_not
    fine_lvl_cosistency = con_sum_f * con_f_not    
   
    ML_c = margin_loss(y_true_c, y_pred_c)*LW_C
    ML_m = LW_M*((1-C_Weight)*(margin_loss(y_true_m, y_pred_m))+C_Weight*(medium_lvl_cosistency))
    ML_f = LW_F*((1-C_Weight)*(margin_loss(y_true_f, y_pred_f))+C_Weight*(fine_lvl_cosistency))
    
    batch_loss = ML_c + ML_m+ ML_f

    return tf.reduce_mean(batch_loss)

## Model Architecture

In [11]:
def get_compiled_model():
    input_shape = dataset['x_train'].shape[1:]

    input_shape_yc = dataset['y_train_coarse'].shape[1:]
    input_shape_ym = dataset['y_train_medium'].shape[1:]
    input_shape_yf = dataset['y_train_fine'].shape[1:]

    no_coarse_class = number_of_classes_c
    no_medium_class = number_of_classes_m
    no_fine_class = number_of_classes_f

    PCap_n_dims = 8

    SCap_f_dims = 8
    SCap_m_dims = 16
    SCap_c_dims = 32


    # Input image
    x_input = keras.layers.Input(shape=input_shape, name="Input_Image")

    # Input True Labels
    y_c = keras.layers.Input(shape=input_shape_yc, name='input_yc')
    y_m = keras.layers.Input(shape=input_shape_ym, name='input_ym')
    y_f = keras.layers.Input(shape=input_shape_yf, name='input_yf')

    #--- block 1 ---
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(x_input)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    #--- block 2 ---
    x = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    #--- block 3 ---
    x = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    #--- block 4 ---
    x = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)


    # Layer 3: Reshape to 8D primary capsules 
    reshapec = keras.layers.Reshape((int((tf.reduce_prod(x.shape[1:]).numpy())/PCap_n_dims),
                                     PCap_n_dims), name="reshape_layer")(x)
    p_caps = keras.layers.Lambda(models.squash, name='p_caps')(reshapec)

    ## Layer Secondary Capsule: For coarse level
    s_caps_c = models.SecondaryCapsule(n_caps=no_coarse_class, n_dims=SCap_c_dims, 
                        name="s_caps_coarse")(p_caps)
    
    ## Layer Secondary Capsule: For medium level
    s_caps_m = models.SecondaryCapsule(n_caps=no_medium_class, n_dims=SCap_m_dims, 
                        name="s_caps_medium")(s_caps_c)

    ## Layer Secondary Capsule: For fine level
    s_caps_f = models.SecondaryCapsule(n_caps=no_fine_class, n_dims=SCap_f_dims, 
                        name="s_caps_fine")(s_caps_m)

    pred_c = models.LengthLayer(name='prediction_coarse')(s_caps_c)

    pred_m = models.LengthLayer(name='prediction_medium')(s_caps_m)

    pred_f = models.LengthLayer(name='prediction_fine')(s_caps_f)

    model = keras.Model(inputs= [x_input, y_c, y_m, y_f],
                        outputs= [pred_c, pred_m, pred_f],
                        name='HD-CapsNet')

    model.add_loss(CustomLoss(y_c, y_m, y_f, pred_c, pred_m, pred_f, 
                              lossweight['coarse_lw'], lossweight['medium_lw'], lossweight['fine_lw'],
                              dataset['tree']))

    model.compile(optimizer='adam',                  
                  metrics={'prediction_fine': 'accuracy',
                           'prediction_medium': 'accuracy',
                           'prediction_coarse': 'accuracy'
                          }
    )
    return model

In [12]:
model = get_compiled_model()

In [13]:
model.summary()
# keras.utils.plot_model(model, to_file = directory+"/Architecture.png", show_shapes=True)

Model: "HD-CapsNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input_Image (InputLayer)       [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 32, 32, 64)   1792        ['Input_Image[0][0]']            
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 64)  256         ['block1_conv1[0][0]']           
 alization)                                                                                       
                                                                                                  
 block1_conv2 (Conv2D)          (None, 32, 32, 64)   36928       ['batch_normalization[0]

                                                                                                  
 tf.cast (TFOpLambda)           (None,)              0           ['tf.math.argmax_4[0][0]']       
                                                                                                  
 tf.reshape (TFOpLambda)        (None, 1)            0           ['tf.math.reduce_sum_2[0][0]']   
                                                                                                  
 tf.math.argmax (TFOpLambda)    (None,)              0           ['prediction_coarse[0][0]']      
                                                                                                  
 tf.math.argmax_1 (TFOpLambda)  (None,)              0           ['prediction_medium[0][0]']      
                                                                                                  
 tf.math.argmax_5 (TFOpLambda)  (None,)              0           ['input_ym[0][0]']               
          

 da)                                                                                              
                                                                                                  
 tf.math.reduce_sum (TFOpLambda  (None,)             0           ['tf.math.multiply[0][0]']       
 )                                                                                                
                                                                                                  
 tf.math.maximum_4 (TFOpLambda)  (None, 100)         0           ['tf.math.subtract_10[0][0]']    
                                                                                                  
 tf.math.subtract_12 (TFOpLambd  (None, 100)         0           ['input_yf[0][0]']               
 a)                                                                                               
                                                                                                  
 tf.math.m

                                                                                                  
 tf.math.multiply_4 (TFOpLambda  (None,)             0           ['tf.math.abs_2[0][0]',          
 )                                                                'tf.math.abs[0][0]']            
                                                                                                  
 tf.math.add_2 (TFOpLambda)     (None, 100)          0           ['tf.math.multiply_16[0][0]',    
                                                                  'tf.math.multiply_18[0][0]']    
                                                                                                  
 tf.math.abs_3 (TFOpLambda)     (None,)              0           ['tf.math.subtract_3[0][0]']     
                                                                                                  
 tf.math.abs_1 (TFOpLambda)     (None,)              0           ['tf.math.subtract_1[0][0]']     
          

## Data Augmentation

In [14]:
datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1)

training_generator = MixUp_add_loss.MixupGenerator_3level(dataset['x_train'],
                                                 dataset['y_train_coarse'], 
                                                 dataset['y_train_medium'],
                                                 dataset['y_train_fine'],
                                                 batch_size=train_params["batch_size"],
                                                 alpha=0.2, 
                                                 datagen=datagen
                                                )()

## Callback

In [15]:
tb = keras.callbacks.TensorBoard(directory+'./tb_logs'+ datetime.now().strftime("%Y%m%d-%H%M%S"))
log = keras.callbacks.CSVLogger(directory+'/log.csv', append=True)

checkpoint = keras.callbacks.ModelCheckpoint(
    directory+'/epoch-best.h5', 
    monitor='val_prediction_fine_accuracy',
    save_best_only=True, save_weights_only=True, verbose=1)
change_lw = models.LossWeightsModifier(lossweight = lossweight,
                               initial_lw = initial_lw,
                               directory = directory)
lr_decay = keras.callbacks.LearningRateScheduler(scheduler)

## Model Training

In [ ]:
model_save_dir = str(directory+'/trained_model.h5')
try:
    model.load_weights(model_save_dir)
except:
    history = model.fit(training_generator,
                        steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                        epochs = train_params["n_epochs"],
                        validation_data = ([dataset['x_test'],
                                            dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']],
                                           [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]),
                        callbacks = [tb,log,change_lw,lr_decay,checkpoint],
                        verbose=1)
    
    model.save_weights(model_save_dir)

Epoch 1/100
781/781 [==============================] - ETA: 0s - loss: 0.5243 - prediction_coarse_accuracy: 0.4603 - prediction_medium_accuracy: 0.1832 - prediction_fine_accuracy: 0.0152
  • Coarse Accuracy = 46.03% | Val_Accuracy = 52.83% | LossWeight = 0.36 
  • Medium Accuracy = 18.32% | Val_Accuracy = 28.49% | LossWeight = 0.49 
  • Fine   Accuracy = 1.52% | Val_Accuracy = 1.71% | LossWeight = 0.15 

Epoch 1: val_prediction_fine_accuracy improved from -inf to 0.01710, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss\epoch-best.h5
781/781 [==============================] - 69s 77ms/step - loss: 0.5243 - prediction_coarse_accuracy: 0.4603 - prediction_medium_accuracy: 0.1832 - prediction_fine_accuracy: 0.0152 - val_loss: 0.4381 - val_prediction_coarse_accuracy: 0.5283 - val_prediction_medium_accuracy: 0.2849 - val_prediction_fine_accuracy: 0.0171 - lr: 0.0010
Epoch 2/100
781/781 [==============================] - ETA: 0s - loss: 0.3620 - prediction_coarse_accuracy:

781/781 [==============================] - ETA: 0s - loss: 0.1973 - prediction_coarse_accuracy: 0.8292 - prediction_medium_accuracy: 0.7265 - prediction_fine_accuracy: 0.1690
  • Coarse Accuracy = 82.92% | Val_Accuracy = 80.06% | LossWeight = 0.28 
  • Medium Accuracy = 72.65% | Val_Accuracy = 68.43% | LossWeight = 0.40 
  • Fine   Accuracy = 16.90% | Val_Accuracy = 16.59% | LossWeight = 0.32 

Epoch 9: val_prediction_fine_accuracy improved from 0.15360 to 0.16590, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss\epoch-best.h5
781/781 [==============================] - 71s 91ms/step - loss: 0.1973 - prediction_coarse_accuracy: 0.8292 - prediction_medium_accuracy: 0.7265 - prediction_fine_accuracy: 0.1690 - val_loss: 0.1831 - val_prediction_coarse_accuracy: 0.8006 - val_prediction_medium_accuracy: 0.6843 - val_prediction_fine_accuracy: 0.1659 - lr: 0.0010
Epoch 10/100
781/781 [==============================] - ETA: 0s - loss: 0.1885 - prediction_coarse_accuracy: 0.840

Epoch 18/100
781/781 [==============================] - ETA: 0s - loss: 0.1359 - prediction_coarse_accuracy: 0.9146 - prediction_medium_accuracy: 0.8696 - prediction_fine_accuracy: 0.2701
  • Coarse Accuracy = 91.46% | Val_Accuracy = 84.04% | LossWeight = 0.23 
  • Medium Accuracy = 86.96% | Val_Accuracy = 73.88% | LossWeight = 0.31 
  • Fine   Accuracy = 27.01% | Val_Accuracy = 23.61% | LossWeight = 0.46 

Epoch 18: val_prediction_fine_accuracy improved from 0.23600 to 0.23610, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss\epoch-best.h5
781/781 [==============================] - 68s 87ms/step - loss: 0.1359 - prediction_coarse_accuracy: 0.9146 - prediction_medium_accuracy: 0.8696 - prediction_fine_accuracy: 0.2701 - val_loss: 0.1508 - val_prediction_coarse_accuracy: 0.8404 - val_prediction_medium_accuracy: 0.7388 - val_prediction_fine_accuracy: 0.2361 - lr: 6.6342e-04
Epoch 19/100
781/781 [==============================] - ETA: 0s - loss: 0.1319 - prediction_coar

Epoch 27/100
781/781 [==============================] - ETA: 0s - loss: 0.1155 - prediction_coarse_accuracy: 0.9401 - prediction_medium_accuracy: 0.9163 - prediction_fine_accuracy: 0.3506
  • Coarse Accuracy = 94.01% | Val_Accuracy = 83.49% | LossWeight = 0.21 
  • Medium Accuracy = 91.63% | Val_Accuracy = 73.42% | LossWeight = 0.26 
  • Fine   Accuracy = 35.06% | Val_Accuracy = 30.23% | LossWeight = 0.53 

Epoch 27: val_prediction_fine_accuracy improved from 0.29590 to 0.30230, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss\epoch-best.h5
781/781 [==============================] - 70s 90ms/step - loss: 0.1155 - prediction_coarse_accuracy: 0.9401 - prediction_medium_accuracy: 0.9163 - prediction_fine_accuracy: 0.3506 - val_loss: 0.1477 - val_prediction_coarse_accuracy: 0.8349 - val_prediction_medium_accuracy: 0.7342 - val_prediction_fine_accuracy: 0.3023 - lr: 4.1812e-04
Epoch 28/100
781/781 [==============================] - ETA: 0s - loss: 0.1144 - prediction_coar

Epoch 36/100
781/781 [==============================] - ETA: 0s - loss: 0.1051 - prediction_coarse_accuracy: 0.9510 - prediction_medium_accuracy: 0.9345 - prediction_fine_accuracy: 0.4209
  • Coarse Accuracy = 95.10% | Val_Accuracy = 85.09% | LossWeight = 0.20 
  • Medium Accuracy = 93.45% | Val_Accuracy = 76.04% | LossWeight = 0.24 
  • Fine   Accuracy = 42.09% | Val_Accuracy = 34.67% | LossWeight = 0.56 

Epoch 36: val_prediction_fine_accuracy improved from 0.34540 to 0.34670, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss\epoch-best.h5
781/781 [==============================] - 64s 82ms/step - loss: 0.1051 - prediction_coarse_accuracy: 0.9510 - prediction_medium_accuracy: 0.9345 - prediction_fine_accuracy: 0.4209 - val_loss: 0.1357 - val_prediction_coarse_accuracy: 0.8509 - val_prediction_medium_accuracy: 0.7604 - val_prediction_fine_accuracy: 0.3467 - lr: 2.6352e-04
Epoch 37/100
781/781 [==============================] - ETA: 0s - loss: 0.1047 - prediction_coar

781/781 [==============================] - ETA: 0s - loss: 0.0998 - prediction_coarse_accuracy: 0.9554 - prediction_medium_accuracy: 0.9422 - prediction_fine_accuracy: 0.4581
  • Coarse Accuracy = 95.54% | Val_Accuracy = 85.45% | LossWeight = 0.20 
  • Medium Accuracy = 94.22% | Val_Accuracy = 76.40% | LossWeight = 0.23 
  • Fine   Accuracy = 45.81% | Val_Accuracy = 37.31% | LossWeight = 0.57 

Epoch 45: val_prediction_fine_accuracy improved from 0.37260 to 0.37310, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss\epoch-best.h5
781/781 [==============================] - 63s 80ms/step - loss: 0.0998 - prediction_coarse_accuracy: 0.9554 - prediction_medium_accuracy: 0.9422 - prediction_fine_accuracy: 0.4581 - val_loss: 0.1313 - val_prediction_coarse_accuracy: 0.8545 - val_prediction_medium_accuracy: 0.7640 - val_prediction_fine_accuracy: 0.3731 - lr: 1.6608e-04
Epoch 46/100
781/781 [==============================] - ETA: 0s - loss: 0.0990 - prediction_coarse_accuracy: 

Epoch 54/100
781/781 [==============================] - ETA: 0s - loss: 0.0972 - prediction_coarse_accuracy: 0.9551 - prediction_medium_accuracy: 0.9419 - prediction_fine_accuracy: 0.4834
  • Coarse Accuracy = 95.51% | Val_Accuracy = 85.76% | LossWeight = 0.21 
  • Medium Accuracy = 94.19% | Val_Accuracy = 76.54% | LossWeight = 0.24 
  • Fine   Accuracy = 48.34% | Val_Accuracy = 39.13% | LossWeight = 0.55 

Epoch 54: val_prediction_fine_accuracy improved from 0.38930 to 0.39130, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss\epoch-best.h5
781/781 [==============================] - 55s 70ms/step - loss: 0.0972 - prediction_coarse_accuracy: 0.9551 - prediction_medium_accuracy: 0.9419 - prediction_fine_accuracy: 0.4834 - val_loss: 0.1287 - val_prediction_coarse_accuracy: 0.8576 - val_prediction_medium_accuracy: 0.7654 - val_prediction_fine_accuracy: 0.3913 - lr: 1.0467e-04
Epoch 55/100
781/781 [==============================] - ETA: 0s - loss: 0.0971 - prediction_coar

Epoch 63/100
781/781 [==============================] - ETA: 0s - loss: 0.0948 - prediction_coarse_accuracy: 0.9576 - prediction_medium_accuracy: 0.9458 - prediction_fine_accuracy: 0.5018
  • Coarse Accuracy = 95.76% | Val_Accuracy = 86.09% | LossWeight = 0.20 
  • Medium Accuracy = 94.58% | Val_Accuracy = 77.07% | LossWeight = 0.24 
  • Fine   Accuracy = 50.18% | Val_Accuracy = 40.22% | LossWeight = 0.56 

Epoch 63: val_prediction_fine_accuracy improved from 0.40040 to 0.40220, saving model to ../../logs/MOD_2/4_CIFAR_100/HD_CapsNet/TD_W_C_loss\epoch-best.h5
781/781 [==============================] - 53s 68ms/step - loss: 0.0948 - prediction_coarse_accuracy: 0.9576 - prediction_medium_accuracy: 0.9458 - prediction_fine_accuracy: 0.5018 - val_loss: 0.1277 - val_prediction_coarse_accuracy: 0.8609 - val_prediction_medium_accuracy: 0.7707 - val_prediction_fine_accuracy: 0.4022 - lr: 6.5971e-05
Epoch 64/100
781/781 [==============================] - ETA: 0s - loss: 0.0954 - prediction_coar

Epoch 72/100
781/781 [==============================] - ETA: 0s - loss: 0.0949 - prediction_coarse_accuracy: 0.9580 - prediction_medium_accuracy: 0.9484 - prediction_fine_accuracy: 0.5151
  • Coarse Accuracy = 95.80% | Val_Accuracy = 86.06% | LossWeight = 0.21 
  • Medium Accuracy = 94.84% | Val_Accuracy = 77.50% | LossWeight = 0.23 
  • Fine   Accuracy = 51.51% | Val_Accuracy = 41.14% | LossWeight = 0.56 

Epoch 72: val_prediction_fine_accuracy did not improve from 0.41180
781/781 [==============================] - 87s 111ms/step - loss: 0.0949 - prediction_coarse_accuracy: 0.9580 - prediction_medium_accuracy: 0.9484 - prediction_fine_accuracy: 0.5151 - val_loss: 0.1276 - val_prediction_coarse_accuracy: 0.8606 - val_prediction_medium_accuracy: 0.7750 - val_prediction_fine_accuracy: 0.4114 - lr: 4.1578e-05
Epoch 73/100
781/781 [==============================] - ETA: 0s - loss: 0.0951 - prediction_coarse_accuracy: 0.9565 - prediction_medium_accuracy: 0.9450 - prediction_fine_accuracy: 0

781/781 [==============================] - ETA: 0s - loss: 0.0944 - prediction_coarse_accuracy: 0.9582 - prediction_medium_accuracy: 0.9458 - prediction_fine_accuracy: 0.5209
  • Coarse Accuracy = 95.82% | Val_Accuracy = 86.13% | LossWeight = 0.21 
  • Medium Accuracy = 94.58% | Val_Accuracy = 77.39% | LossWeight = 0.24 
  • Fine   Accuracy = 52.09% | Val_Accuracy = 41.71% | LossWeight = 0.55 

Epoch 81: val_prediction_fine_accuracy did not improve from 0.41810
781/781 [==============================] - 52s 67ms/step - loss: 0.0944 - prediction_coarse_accuracy: 0.9582 - prediction_medium_accuracy: 0.9458 - prediction_fine_accuracy: 0.5209 - val_loss: 0.1265 - val_prediction_coarse_accuracy: 0.8613 - val_prediction_medium_accuracy: 0.7739 - val_prediction_fine_accuracy: 0.4171 - lr: 2.6205e-05
Epoch 82/100
781/781 [==============================] - ETA: 0s - loss: 0.0947 - prediction_coarse_accuracy: 0.9565 - prediction_medium_accuracy: 0.9453 - prediction_fine_accuracy: 0.5195
  • Coar

In [ ]:
try:
    history_dict = history.history

    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({"Coarse": history}, metric = "prediction_coarse_accuracy")
    plotter.plot({"Medium": history}, metric = "prediction_medium_accuracy")
    plotter.plot({"Fine": history}, metric = "prediction_fine_accuracy")
    plt.title("Model Accuracy")
    plt.ylim([0,1])
except:
    print('Trained model weights loaded')

In [ ]:
try:
    plotter = tfdocs.plots.HistoryPlotter()
    plotter.plot({"loss": history}, metric = "loss")
    plt.title("Model Loss")
    plt.ylim([0,1])
except:
    print('Trained model weights loaded')

## Model Analysis

In [ ]:
final_model = keras.Model(model.inputs[:1], model.output)

lossfn = models.MarginLoss()
final_model.compile(optimizer='adam', 
                    loss={'prediction_fine' : lossfn,
                          'prediction_medium' : lossfn,
                          'prediction_coarse' : lossfn},

                    loss_weights={'prediction_fine' : lossweight['fine_lw'],
                                  'prediction_medium' : lossweight['medium_lw'],
                                  'prediction_coarse' : lossweight['coarse_lw']},

                    metrics={'prediction_fine': 'accuracy',
                             'prediction_medium': 'accuracy',
                             'prediction_coarse': 'accuracy'
                            }
                   )

In [ ]:
model_analysis = models.model_analysis(final_model, dataset)
results = model_analysis.evaluate()
predictions = model_analysis.prediction()

In [ ]:
true_label = [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1],predictions[2]]
metrics.lvl_wise_metric(true_label,pred_label)

In [ ]:
h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\nConsistency = ', consistency,
      '\nExact Match = ', exact_match,
     )

# Training Again

In [ ]:
model_save_dir = str(directory+'/trained_model_2.h5')
try:
    model.load_weights(model_save_dir)
except:
    history = model.fit(training_generator,
                        steps_per_epoch = int(dataset['x_train'].shape[0] / train_params["batch_size"]),
                        epochs = train_params["n_epochs"],
                        validation_data = ([dataset['x_test'],
                                            dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']],
                                           [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]),
                        callbacks = [tb,log,change_lw,lr_decay,checkpoint],
                        verbose=1)
    
    model.save_weights(model_save_dir)

In [ ]:
history_dict = history.history

In [ ]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({"Coarse": history}, metric = "prediction_coarse_accuracy")
plotter.plot({"Medium": history}, metric = "prediction_medium_accuracy")
plotter.plot({"Fine": history}, metric = "prediction_fine_accuracy")
plt.title("Model Accuracy")
plt.ylim([0,1])

In [ ]:
plotter = tfdocs.plots.HistoryPlotter()
plotter.plot({"loss": history}, metric = "loss")
plt.title("Model Loss")
plt.ylim([0,1])

## Model Analysis

In [ ]:
final_model = keras.Model(model.inputs[:1], model.output)

lossfn = models.MarginLoss()
final_model.compile(optimizer='adam', 
                    loss={'prediction_fine' : lossfn,
                          'prediction_medium' : lossfn,
                          'prediction_coarse' : lossfn},

                    loss_weights={'prediction_fine' : lossweight['fine_lw'],
                                  'prediction_medium' : lossweight['medium_lw'],
                                  'prediction_coarse' : lossweight['coarse_lw']},

                    metrics={'prediction_fine': 'accuracy',
                             'prediction_medium': 'accuracy',
                             'prediction_coarse': 'accuracy'
                            }
                   )

In [ ]:
model_analysis = models.model_analysis(final_model, dataset)
results = model_analysis.evaluate()
predictions = model_analysis.prediction()

In [ ]:
true_label = [dataset['y_test_coarse'],dataset['y_test_medium'],dataset['y_test_fine']]
pred_label = [predictions[0],predictions[1],predictions[2]]
metrics.lvl_wise_metric(true_label,pred_label)

In [ ]:
h_measurements,consistency,exact_match = metrics.hmeasurements(true_label,
                                       pred_label,
                                       dataset['tree'])
print('\nHierarchical Precision =',h_measurements[0],
      '\nHierarchical Recall =', h_measurements[1],
      '\nHierarchical F1-Score =',h_measurements[2],
      '\nConsistency = ', consistency,
      '\nExact Match = ', exact_match,
     )